In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import one_hot
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
import numpy as np

from dataloader import train_dataloader, val_dataloader, SubjectDataset, train_dataset
from model import BRNN, OneDConvNet
from torch.utils.data import Dataset, DataLoader

In [2]:
from torch.nn import functional as F

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'mps'
input_size = 6
sequence_length = 335413
num_layers = 2
hidden_size = 125
num_classes = 4
learning_rate = 0.0001
batch_size = 128
num_epochs = 40

In [4]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

def train_step(X, y, model, optimizer, criterion):
    y = F.one_hot(y, 4).float()
    y_pred = model(X)
    predicted_classes = torch.argmax(y_pred.detach(), dim=1)

    loss = criterion(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    corrects = accuracy_fn(y.argmax(dim=1), y_pred.argmax(dim=1))

    return loss.item(), corrects

def val_step(X, y, model, criterion):

    with torch.no_grad():
        y = F.one_hot(y, 4).float()
        y_pred = model(X)
        predicted_classes = torch.argmax(y_pred.detach(), dim=1)
        loss = criterion(y_pred, y)
        corrects = accuracy_fn(y.argmax(dim=1), y_pred.argmax(dim=1))

    return loss.item(), corrects, predicted_classes.detach().cpu().numpy()

In [7]:
model = OneDConvNet(input_size, num_classes, 8, 120).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=1, verbose=True)

In [7]:
model = BRNN(6,150,1,4,device).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=1, verbose=True)

In [8]:
best_val_loss = -10000000
for epoch in range(num_epochs):
    # Train for "n" number of iterations
    running_loss = 0.
    running_acc = 0.
    for iteration, (X, y) in enumerate(train_dataloader):

        X = X.float().to(device)
        # Normalize
        # X = (X - min) / (max - min)
        # X = torch.transpose(X, 1, 2)
        y = y.to(device)

        loss, corrects = train_step(X, y, model, optimizer, criterion)

        # Running metrics
        running_loss = running_loss + loss 
        running_acc = running_acc + corrects

        

    train_loss = running_loss / len(train_dataloader)
    train_acc = running_acc / len(train_dataloader)

    # Validate
    running_val_loss = 0.
    running_val_acc = 0.
    for step, (X, y) in enumerate(val_dataloader):

        X = X.float().to(device)
        # X = (X - min) / (max - min)
        # X = torch.transpose(X, 1, 2)
        y = y.to(device)

        loss, corrects, predicted_classes = val_step(X, y, model, criterion)
        # Running metrics
        running_val_loss = running_val_loss + loss
        running_val_acc = running_val_acc + corrects

    val_loss = running_val_loss / len(val_dataloader)
    val_acc = running_val_acc / len(val_dataloader)

    # scheduler.step(val_loss)

    # if val_loss < best_val_loss:
    #     # Checkpoint model
    #     path = "checkpoint_model_lstm_filtered_run_1.pth"
    #     print(f"Saving model to {path}")
    #     torch.save(model.state_dict(), path)
    #     best_val_loss = val_loss

    print(f"Epoch: {epoch} | train_loss {train_loss} | train_acc: {train_acc} | val_loss: {val_loss} | val_acc: {val_acc}")

Epoch: 0 | train_loss 1.3929558517411351 | train_acc: 24.859619140625 | val_loss: 1.3815879718117092 | val_acc: 26.919157608695652
Epoch: 1 | train_loss 1.3914476800709963 | train_acc: 24.969482421875 | val_loss: 1.3789400251015373 | val_acc: 27.61548913043478
Epoch: 2 | train_loss 1.3913907641544938 | train_acc: 24.688720703125 | val_loss: 1.3810309171676636 | val_acc: 27.156929347826086
Epoch: 3 | train_loss 1.3908512042835355 | train_acc: 24.90234375 | val_loss: 1.384506546932718 | val_acc: 24.91508152173913
Epoch: 4 | train_loss 1.389975962229073 | train_acc: 25.750732421875 | val_loss: 1.384939432144165 | val_acc: 25.322690217391305
Epoch: 5 | train_loss 1.389925392344594 | train_acc: 25.164794921875 | val_loss: 1.3852243734442669 | val_acc: 25.152853260869566
Epoch: 6 | train_loss 1.3892512619495392 | train_acc: 25.067138671875 | val_loss: 1.386181502238564 | val_acc: 24.91508152173913


KeyboardInterrupt: 

In [27]:
model = TCN(6, 4).to(device)

In [66]:
model = BRNN(6, 150, 2, 4, device).to(device)

In [67]:
model.load_state_dict(torch.load('/Users/purushothamanyadav/Documents/NCSU/Spring23/NN/Project/ProjC/terrain-identification/checkpoint_model_lstm_run_20.pth', map_location=torch.device('mps')))

<All keys matched successfully>

In [10]:
from tsai.models import TransformerModel

In [18]:
bs = batch_size
nvars = 6
seq_len = 120
c_out = 4
xb = torch.rand(bs, nvars, seq_len)


model = TransformerModel(nvars, c_out, d_model=64, n_head=1, d_ffn=128, dropout=0.1, activation='gelu', n_layers=3)
# test_eq(model(xb).shape, [bs, c_out])
# print(count_parameters(model))
model.to(device)

TransformerModel(
  (permute): Permute(dims=2, 0, 1)
  (inlinear): Linear(in_features=6, out_features=64, bias=True)
  (relu): ReLU()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (transpose): Transpose(1, 0)
  (max): Max(dim=1, keepdim=False)
  (outlinear): Linear(in

In [14]:
from tsai.all import *

In [15]:
TransformerModel()

TypeError: TransformerModel.__init__() missing 2 required positional arguments: 'c_in' and 'c_out'

In [68]:
X.shape

torch.Size([128, 6, 40])

In [69]:
import pandas as pd
FILENAME_TEMPLATE = "subject_{}_{}__y.csv"

In [70]:
save_dir = "/Users/purushothamanyadav/Documents/NCSU/Spring23/NN/Project/ProjC/terrain-identification/predictions/C3.2/LSTM"

In [65]:
test_data_path = "/Users/purushothamanyadav/Documents/NCSU/Spring23/NN/Project/ProjC/terrain-identification/data/TestData/window_3"
split_ids = ["009_01", "010_01", "011_01", "012_01"]
batch_size = 128
model.eval()
for id in split_ids:

    test_dataset_1sec = SubjectDataset(
        test_data_path, 
        [id]
    )
    test_dataloader_1sec = DataLoader(test_dataset_1sec, batch_size=batch_size, shuffle=False)
    

    
    

    output = []

    for (X_1sec, y_1sec) in test_dataloader_1sec:

        X_1sec = X_1sec.float().to(device)
        X_1sec = (X_1sec - min) / (max - min)

        X_1sec = torch.transpose(X_1sec, 1, 2)
        if X_1sec.shape[0] != 128:
            X_1sec = torch.cat((X_1sec, torch.zeros((128-X_1sec.shape[0], 120, 6)).to(device)), dim=0)
        y_pred = model(X_1sec)
        

       
        predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()

        output.append(predicted_classes)

    _output = np.concatenate(output, axis=0)

    df = pd.DataFrame({"label": _output})

    subject_id, session_id = id.split("_")

    filename = FILENAME_TEMPLATE.format(subject_id, session_id)
    df.to_csv(os.path.join(save_dir, filename), header=False, index=False)

In [40]:
X_1sec.shape

torch.Size([26, 120, 6])

In [62]:
torch.cat((X_1sec, torch.zeros((128-26, 120, 6)).to(device)), dim=0).shape

torch.Size([128, 120, 6])

In [52]:
F.pad(X_1sec, (0,128-26,0,0)).shape

torch.Size([26, 120, 108])

In [41]:
ggg = next(iter(test_dataloader_1sec))

In [43]:
ggg[0].shape

torch.Size([128, 6, 120])